## Customização de palavras no Watson Text to Speech

Se você já testou alguma solução de mercado para a sintetização de áudio a partir de texto (text to speech) então você provavelmente já se deparou com alguma palavra que soou errado, isso ocorre porque as solução de fala para texto (TTS) são treinadas considerando o português comum que encontramos no dicionário.

No entanto é muito comum encontrarmos siglas, termos específicos ou até mesmo a utilização de palavras estrangeiras dentro de uma empresa nas nossas respostas. Como podemos adaptar nosso sistema cognitivo para que possa suportar tais especializações?

Um método seria escrever a palavra errada em nossas respostas para que então ela fosse vocalizada corretamente nos sistemas que utilizam áudio, mas essa abordagem nos tira a flexibilidade de poder treinar um mesmo corpus e permitir a utilização tanto por texto quanto por voz.

Nesse tutorial vamos apresentar como customizar o Watson Text to Speech de modo que ele fale as suas palavras corretamente em português.

### ***"Um dos diferenciais do serviço Text to Speech da IBM é a possibilidade de customizar como essas palavras são expressas (em português)."***

* Autor: Renato Leal
* Medium: http://medium.com/@renatoleal
* Linkedin: https://www.linkedin.com/in/renatodossantosleal/
* Github: https://github.com/renatodossantosleal

**Etapas**

* [Blibliotecas](#bibliotecas)
* [Credenciais](#credenciais)
* [Teste de sintetização](#teste_sint)
* [Cria um modelo novo do seu modelo de customização](#custom)
* [Descritivo do modelo (opcional)](#descritivo)
* [Adiciona palavras ao modelo](#adiciona)
* [Testar novamente a sintetização](#testa_novamente)


**OBS: utilizaremos python nesse notebook mas você facilmente pode fazer isso em nodeJS, java, swift e outras linguagens. Nos comentários também deixarei os comandos em cURL**

# Antes de começar

**Este tutorial foi criado utilizando o Watson Studio (https://dataplatform.ibm.com/registration/stepone?context=wdp&apps=data_science_experience), caso você queira rodar o mesmo com seu próprio serviço e termos basta criar uma conta no IBM Cloud e instanciar um serivço de Watson Studio (gratuito para um usuário e até 5GB de armazenamento).**

Com o serviço já disponível basta criar um projeto e adicionar um jupyter notebook utilizando a opção de importar a partir de uma URL (o ambiente utilizado foi o gratuito de Anaconda com Python 3.5).

Se você tiver dificuldade com esse passo então esse tutorial aqui pode te ajudar: https://datascience.ibm.com/docs/content/analyze-data/creating-notebooks.html

### <a id="bibliotecas"></a>Bibliotecas

Aqui utilizaremos o SDK de Watson como acelerador para as chamadas no TTS, também importamos o `IPython` para facilitar a reprodução de áudio e `json` para a manipulação de arquivos deste tipo.

OBS: se você ainda não instalou o sdk do Watson no seu ambiente então pode fazê-lo rodando uma nova célula com `#!pip install --upgrade watson-developer-cloud` (sem o hashtag #)

In [1]:
#!pip install watson_developer_cloud

import json
import IPython
from watson_developer_cloud import TextToSpeechV1

### <a id="credenciais"></a>Credenciais

Insere credenciais no código e instancia um objeto de Watson.

OBS: nunca compartilhe duas credenciais no código, caso vá compartilhar esse notebook então adicione a tag "# @hidden_cell" na celula que possui suas credenciais (sem aspas).

In [2]:
text_to_speech = TextToSpeechV1(username='username',password='password')

In [3]:
# The code was removed by DSX for sharing.

### <a id="teste_sint"></a>Teste de sintetização

Você já tentou reproduzir uma palavra em inglês quando configurou o seu text-to-speech em português? Vamos fazer um teste com a palavra "text-to-speech".

Precisamos passar os seguintes parâmetros para o Watson:
* `text` é a frase/palavra a ser sintetizada.
* `accept` é o tipo de arquivo a ser retornado (formato do áudio).
* `voice` é a voz que utilizaremos, para português devemos especificar o valor `pt-BR_IsabelaVoice`

O mesmo comando em cURL: 

`curl -X GET -u "{username}":"{password}" --output exemplo_audio_1.wav "https://stream.watsonplatform.net/text-to-speech/api/v1/synthesize?accept=audio/wav&text=text-to-speech&voice=pt-BR_IsabelaVoice"`

In [4]:
with open("exemplo_audio_1.wav",'wb') as audio_file:
    audio_file.write(text_to_speech.synthesize('text-to-speech', accept='audio/wav',voice="pt-BR_IsabelaVoice").content)
    
IPython.display.Audio("exemplo_audio_1.wav")

### <a id="custom"></a>Cria um modelo novo do seu modelo de customização

Para corrigir a pronúncia do nosso TTS devemos criar um novo modelo de customização e para isso devemos chamar o método `create_voice_model` passando os seguintes parâmetros:

* `name` é o nome do modelo a ser criado, você pode especificar o nome que quiser.
* `language` deverá ser `pt-BR` para que o modelo seja utilizado considerando o português do Brasil.
* `description` é opicional e permite que você coloque mais informações sobre o modelo.

A resposta dessa chamada será um `json` no qual devemos salvar o valor do campo `customization_id` que será utilizado depois.

O mesmo comando em cURL: 

`curl -X POST -u "{username}":"{password}" --header "Content-Type: application/json" --data "{\"name\":\"medium-custom_2\", \"language\":\"pt-BR\", \"description\":\"\"}" "https://stream.watsonplatform.net/text-to-speech/api/v1/customizations"`

In [5]:
obj_customization = text_to_speech.create_voice_model(name='medium-custom_2',language="pt-BR",description="")
customization_id = obj_customization["customization_id"]

print("Id de customização: " + customization_id)

Id de customização: bb12b13e-0d20-42a2-ba65-55dca975616a


#### <a id="descritivo"></a>Descritivo do modelo (opcional)
Podemos também verificar todas as informações presentes no modelo que acabamos de criar chamando o método `get_voice_model` e passando o `customization_id`:

O mesmo comando em cURL:

`curl -X GET -u "{username}":"{password}" "https://stream.watsonplatform.net/text-to-speech/api/v1/customizations/6af96480-e8dd-4087-aa27-9e07d5b075c9"`

In [6]:
print(json.dumps(text_to_speech.get_voice_model(customization_id),indent=2))

{
  "description": "",
  "last_modified": "2018-03-29T17:42:20.123Z",
  "language": "pt-BR",
  "created": "2018-03-29T17:42:20.123Z",
  "customization_id": "bb12b13e-0d20-42a2-ba65-55dca975616a",
  "words": [],
  "name": "medium-custom_2",
  "owner": "e4bd7023-d183-4560-892a-1970684451d7"
}


### <a id="adiciona"></a>Adiciona palavras ao modelo

Depois que criamos o modelo de customização basta adicionar as palavras e suas respectivas transcrições ao modelo, para isso podemos fazer palavra por palavra ou então por "batch" (não faremos aqui mas você apenas precisaria montar um arquivo json e submeter o mesmo para o serviço do TTS usando o método `add_words`). 

No caso de palavra por palavra devemos chamar o método `add_word` com o seguintes parâmetros:
* `customization_id` é o id do seu modelo de customização.
* `word` é a palavra que hoje possui uma sintetização problemática.
* `translation` é o como a palavra deveria soar. **\[1\]**

**\[1\] A parte de translation você pode realizar de duas formas diferentes:**
* **Sounds-like**: como a palavra deveria soar? Aqui fazemos a tradução utilizando similaridade.
* **Phonetic**: trabalhamos em como descrever a pronúncia de uma palavra através de fonemas, para isso você provavelmente precisará de um linguista mas também poderá atingir resultados mais corretos.

Aqui utilizaremos apenas a customização via ***sounds-like***, caso você queira realizar via fonemas basta acessar este link: https://console.bluemix.net/docs/services/text-to-speech/custom-intro.html#phonetic.

O mesmo comando em cURL:

`curl -X PUT -u "{username}":"{password}" --header "Content-Type: application/json" --data "{\"translation\":\"técsti tchu spit\"}" "https://stream.watsonplatform.net/text-to-speech/api/v1/customizations/6af96480-e8dd-4087-aa27-9e07d5b075c9/words/text-to-speech"`

In [7]:
# Exemplo para adicionar apenas uma palavra
print(text_to_speech.add_word(customization_id,word='text-to-speech',translation='técsti tchu spit'))

None


### <a id="testa_novamente"></a>Testar novamente a sintetização

Assim como fizemos antes, vamos chamar o serviço do TTS para rever a sintetização após essas alterações. A única diferença é que agora devemos adicionar o parâmetro `customization_id` a chamada.

O mesmo comando em cURL:

`curl -X GET -u "{username}":"{password}" --output exemplo_audio_1.wav "https://stream.watsonplatform.net/text-to-speech/api/v1/synthesize?accept=audio/wav&text=text-to-speech&voice=pt-BR_IsabelaVoice&customization_id=6af96480-e8dd-4087-aa27-9e07d5b075c9"`

In [8]:
with open("exemplo_audio_2.wav",'wb') as audio_file:
    audio_file.write(text_to_speech.synthesize('text-to-speech', accept='audio/wav',voice="pt-BR_IsabelaVoice", customization_id=customization_id).content)
    
IPython.display.Audio("exemplo_audio_2.wav")

## Conclusão

Vimos aqui que apenas com duas chamadas (`create_voice_model` e `add_word`) **ao Text to Speech da IBM podemos customizá-lo para o português do Brasil**, essa possibilidade é de extrema importância para que possamos criar nosso próprio vocabulário e a vocalização de tais palavras da maneira correta. 